## Foundations of Computer Science

### Marco Ferrario 795203

In [1]:
import numpy as np
import pandas as pd
import random
import time

In [2]:
start = time.time()

In [3]:
loans = pd.read_csv("loans.csv")

In [4]:
lenders = pd.read_csv("lenders.csv")

In [5]:
loans_lenders = pd.read_csv("loans_lenders.csv")

### 1) Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.

In [6]:
loans_lenders

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."
...,...,...
1387427,678999,"michael43411218, carol5987, gooddogg1, chris41..."
1387428,1207353,"rjhoward1986, jeffrey6870, trolltech4460, elys..."
1387429,1206220,"vicky7746, gooddogg1, fairspirit, craig9729960..."
1387430,1206425,"rich6705, sergiiy9766, angela7509, barbara5610..."


Per normalizzare la tabella loans_lenders si utilizzano ```str.split``` ed ```explode```.

In [7]:
#punto 1 modo b
loans_lenders = loans_lenders.assign(lenders=loans_lenders.lenders.str.split(',')).explode('lenders').reset_index(drop=True)

Controllo il numero di valori unici per `loan_id` e `lenders`.

In [8]:
loans_lenders.loan_id.nunique()

1387432

In [9]:
loans_lenders.lenders.nunique()

1639026

La tabella normalizzata contiene alcune righe duplicate, questo perchè alcune persone hanno partecipato più volte allo stesso prestito.

In [10]:
duplicateRowsDF = loans_lenders[loans_lenders.duplicated()]
 
print("Righe duplicate del dataset:")
print(duplicateRowsDF)

Righe duplicate del dataset:
          loan_id        lenders
45         483738   danhostetler
46         483738   danhostetler
121        563395   motoharu1020
128        575414     dougal1825
152        578029       paul1853
...           ...            ...
28293582  1204948           yipe
28293595  1204948     jordan2934
28293703  1187825       hans8594
28293704  1187825       hans8594
28293705  1187825       hans8594

[796126 rows x 2 columns]


In [11]:
loans_lenders.isnull().sum()

loan_id    0
lenders    0
dtype: int64

Elimino i duplicati.

In [12]:
loans_lenders_unique = loans_lenders.drop_duplicates()

In [13]:
loans_lenders_unique

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


### 2) For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

In [14]:
#punto 2
loans['disburse_time'] = pd.to_datetime(loans['disburse_time'])
loans['planned_expiration_time'] = pd.to_datetime(loans['planned_expiration_time'])

loans['disburse_date'] = loans['disburse_time'].dt.date
loans['planned_expiration_date'] = loans['planned_expiration_time'].dt.date

Attraverso le funzioni `to_date` e `to_datetime` è possibile effettuare la differenza tra le colonne in esame.

In [15]:
loans['duration'] = loans['planned_expiration_date'] - loans['disburse_date'] 

In [16]:
loans[['disburse_time','planned_expiration_time','duration']].isnull().sum()

disburse_time                2813
planned_expiration_time    371834
duration                   374645
dtype: int64

Verifica della quantià dei missing. Si nota che due righe presentano valori mancanti sia per `disburse_time` che per `planned_expiration_time`.

In [17]:
loans['duration']

0         54 days
1         96 days
2         37 days
3         35 days
4         58 days
            ...  
1419602   40 days
1419603   39 days
1419604   51 days
1419605   63 days
1419606   61 days
Name: duration, Length: 1419607, dtype: timedelta64[ns]

Si osserva che la durata media è di 51 giorni, con un massimo di 1674 giorni.

In [18]:
loans['duration'].describe()

count                    1044962
mean     51 days 19:56:30.592959
std      29 days 14:51:03.489556
min          -138 days +00:00:00
25%             42 days 00:00:00
50%             52 days 00:00:00
75%             62 days 00:00:00
max           1674 days 00:00:00
Name: duration, dtype: object

### 3) Find the lenders that have funded at least twice.

In [19]:
#punto 3
lenders_names = loans_lenders.groupby(['lenders'], as_index=False).count()
lenders_twice = lenders_names.loc[(lenders_names['loan_id'] >= 2)]
lenders_twice

,lenders,loan_id
0,000,39
1,00000,39
2,0002,70
4,0101craign0101,71
5,0132575,4
...,...,...
1639018,zyrorl,3
1639020,zzaman,11
1639021,zzanita,2
1639024,zzmcfate,56


Dopo aver raggruppato per `lenders` si selezionano quelli che hanno effettuato almeno due prestiti, risultati 1005111.

### 4) For each country, compute how many loans have involved that country as borrowers.

In [20]:
#punto 4
pd.set_option('display.max_columns', 500)
loans[0:10000]

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,disburse_date,planned_expiration_date,duration
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,to buy additional inventories for her sari-sa...,PH,Philippines,Ozamiz - Ozamiz City,shared,0.1,PHP,126.0,2014-01-15 02:23:45.000 +0000,2014-02-14 03:30:06+00:00,2013-12-22 08:00:00+00:00,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner,2013-12-22,2014-02-14,54 days
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,"To buy American clothing such as blouses, shir...",HN,Honduras,"La Lopez, Choloma",shared,0.1,HNL,201.0,2014-01-14 20:23:20.000 +0000,2014-03-26 22:25:07+00:00,2013-12-20 08:00:00+00:00,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner,2013-12-20,2014-03-26,96 days
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,"to buy stock of rice, sugar and flour",PK,Pakistan,Lala Musa,shared,0.1,PKR,245.0,2014-01-16 11:32:58.000 +0000,2014-02-15 21:10:05+00:00,2014-01-09 08:00:00+00:00,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,2014-01-09,2014-02-15,37 days
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,"to buy cucumber and tomato seeds, as well as f...",KG,Kyrgyzstan,"Aravan village, Osh region",shared,0.1,KGS,171.0,2014-01-20 09:59:48.000 +0000,2014-02-21 03:10:02+00:00,2014-01-17 08:00:00+00:00,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner,2014-01-17,2014-02-21,35 days
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,to buy organic fertilizer and agrochemical pr...,PH,Philippines,"Baleleng, Sto. Thomas, Isabela",shared,0.1,PHP,123.0,2014-01-14 05:46:21.000 +0000,2014-02-13 06:10:02+00:00,2013-12-17 08:00:00+00:00,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,2013-12-17,2014-02-13,58 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,996964,Pauline's Group,English,"Pauline is a 42-year-old, proud mother of six ...","Pauline is a 42-year-old, proud mother of six ...",350.0,350.0,funded,Farming,Agriculture,to purchase a solar light and gain access to c...,KE,Kenya,Bungoma South,shared,0.1,KES,202.0,2015-12-18 02:33:26.000 +0000,2016-01-20 17:20:13+00:00,2016-02-01 08:00:00+00:00,2015-12-23 15:38:46.000 +0000,11.0,14,2,2,NaN,"female, female, female, male, male, female, fe...","true, true, true, true, true, true, true, true",bullet,field_partner,2016-02-01,2016-01-20,-12 days
9996,997380,Amivi massanvi,French,"\nMariée et mère de deux enfants, Amivi est un...","The married mother of two, Amivi is 33 years o...",150.0,150.0,funded,Cloth & Dressmaking Supplies,Retail,to purchase 8 lengths of 'pagne' fabrics,TG,Togo,Tokoin,shared,0.1,XOF,296.0,2015-12-18 16:07:27.000 +0000,2016-01-24 00:50:04+00:00,2015-12-04 08:00:00+00:00,2015-12-25 02:51:17.000 +0000,7.0,5,1,1,NaN,female,true,irregular,field_partner,2015-12-04,2016-01-24,51 days
9997,997880,Mariamu,English,"Mariamu is 27 years old, and she is curre

In [21]:
loans['country_name'].isnull().sum()

0

In [22]:
loans['loan_id'].nunique()

1419607

In [23]:
loans['country_name'].value_counts()

Philippines         285336
Kenya               143699
Peru                 86000
Cambodia             79701
El Salvador          64037
                     ...  
Mauritania               1
Botswana                 1
Uruguay                  1
Papua New Guinea         1
Canada                   1
Name: country_name, Length: 96, dtype: int64

Attraverso la funzione `value_count` si osservano i prestiti effettuati per ogni stato. Si osserva che le Filippine ne hanno ricevuti la quantità maggiore.

In [24]:
loans['country_name'].value_counts().describe()

count        96.000000
mean      14787.572917
std       34997.602451
min           1.000000
25%         477.500000
50%        4667.500000
75%       15110.500000
max      285336.000000
Name: country_name, dtype: float64

### 5) For each country, compute the overall amount of money borrowed

In [25]:
#punto 5
loans[['loan_amount']].isnull().sum()

loan_amount    0
dtype: int64

In [26]:
loans[['country_name']].isnull().sum()

country_name    0
dtype: int64

In [27]:
total_amount = loans.groupby('country_name', as_index=False)['loan_amount'].sum()
total_amount

,country_name,loan_amount
0,Afghanistan,1967950.0
1,Albania,4307350.0
2,Armenia,22950475.0
3,Azerbaijan,14784625.0
4,Belize,150175.0
...,...,...
91,Vietnam,24681100.0
92,Virgin Islands,10000.0
93,Yemen,3444000.0
94,Zambia,1978975.0


Viene effettuato un raggruppamento per `country_name` e sommato il valore del prestito.

In [28]:
total_amount['loan_amount'].describe()

count    9.600000e+01
mean     1.230664e+07
std      1.821081e+07
min      5.000000e+03
25%      7.139250e+05
50%      4.133150e+06
75%      1.611655e+07
max      9.798460e+07
Name: loan_amount, dtype: float64

Lo stato con il maggior numero di denaro prestato sono le Filippine, con oltre 97 milioni di dollari, mentre quello con il minor numero è Gaza, con solo 5000 dollari di prestito.

In [29]:
total_amount.loc[total_amount['loan_amount'].idxmax()]

country_name    Philippines
loan_amount     9.79846e+07
Name: 66, dtype: object

In [30]:
total_amount.loc[total_amount['loan_amount'].idxmin()]

country_name    Gaza
loan_amount     5000
Name: 28, dtype: object

### 6) Like the previous point, but expressed as a percentage of the overall amount lent

In [31]:
# punto 6
total_amount['loan_amount'] = total_amount['loan_amount']/total_amount['loan_amount'].sum()*100
total_perc = total_amount.sort_values('loan_amount', ascending=False)
total_perc

,country_name,loan_amount
66,Philippines,8.293678
65,Peru,6.723825
40,Kenya,5.648711
64,Paraguay,4.567716
14,Cambodia,4.368706
...,...,...
92,Virgin Islands,0.000846
90,Vanuatu,0.000783
9,Botswana,0.000677
89,Uruguay,0.000677


Per calcolare la percentuale si divide il valore di ogni cella per il totale del prestito moltiplicato per 100.

### 7) Like the three previous points, but split for each year (with respect to disburse time).

In [32]:
# punto 7
loans['disburse_year'] = loans['disburse_time'].dt.year

anno = loans[['country_name','disburse_year','loan_amount']]
anno = anno.dropna()

anno['disburse_year'] = anno['disburse_year'].astype(int)

Come osservato nel punto 2 la colonna `disburse_year` presenta dei valori mancanti. Le relative righe non possono essere prese in considerazione per svolgere questo esercizio. Viene utilizzata nuovamente la funzione `groupby`.

In [33]:
total_count = anno.groupby(['country_name','disburse_year']).size()
total_count = pd.DataFrame(total_count)
total_count[0]

country_name  disburse_year
Afghanistan   2007              408
              2008              370
              2009              678
              2010              632
              2011              247
                               ... 
Zimbabwe      2013              426
              2014             2078
              2015              600
              2016              808
              2017             1079
Name: 0, Length: 748, dtype: int64

In [34]:
total_year = anno.groupby(['country_name','disburse_year'])['loan_amount'].sum()
total_year = pd.DataFrame(total_year)
total_year['loan_amount']

country_name  disburse_year
Afghanistan   2007              194975.0
              2008              365375.0
              2009              585125.0
              2010              563350.0
              2011              245125.0
                                 ...    
Zimbabwe      2013              678525.0
              2014             1311575.0
              2015              723625.0
              2016              788600.0
              2017             1237600.0
Name: loan_amount, Length: 748, dtype: float64

In [35]:
total_year['loan_amount'] = total_year['loan_amount']/total_year['loan_amount'].sum()*100
total_year['loan_amount']

country_name  disburse_year
Afghanistan   2007             0.016657
              2008             0.031215
              2009             0.049989
              2010             0.048129
              2011             0.020942
                                 ...   
Zimbabwe      2013             0.057969
              2014             0.112053
              2015             0.061822
              2016             0.067373
              2017             0.105733
Name: loan_amount, Length: 748, dtype: float64

In [36]:
total_year['loan_amount'].sum() # verifica che il totale sia uguale a 100

100.0

### 8) For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

In [37]:
# punto 8
loans_amount = loans[['loan_id','loan_amount','num_lenders_total']]

Si uniscono il dataset loan_lenders normalizzato e il sottoinsieme di loans, contenente l'id e l'importo del prestito.

In [38]:
loanlen_amount = pd.merge(loans_amount, loans_lenders, on='loan_id')

Non conoscendo la quantità di denaro investito di ogni lenders, si divide l'ammontare per il numero di partecipanti al prestito.

In [39]:
loanlen_amount['amount'] = loanlen_amount['loan_amount']/loanlen_amount.groupby('loan_id').loan_id.transform('count')

In [40]:
loanlen_amount['amount_2'] = loanlen_amount['loan_amount']/loanlen_amount['num_lenders_total']

In [41]:
loanlen_amount

,loan_id,loan_amount,num_lenders_total,lenders,amount,amount_2
0,657307,125.0,3,spencer5657,41.666667,41.666667
1,657307,125.0,3,matthew8640,41.666667,41.666667
2,657307,125.0,3,larry71496105,41.666667,41.666667
3,657259,400.0,11,ltr,57.142857,36.363636
4,657259,400.0,11,andrew5306,57.142857,36.363636
...,...,...,...,...,...,...
28293907,989240,175.0,7,jensdamsgaardvanar,35.000000,25.000000
28293908,989240,175.0,7,david47598776,35.000000,25.000000
28293909,989240,175.0,7,bernd9221,35.000000,25.000000
28293910,989240,175.0,7,valeria98599473,35.000000,25.000000


Si raggruppa per lenders e si somma la quantità prestata. Sono riportati due risultati diversi, uno ottenuto dividendo per il numero di lenders in riferimento ai `loan_id` presenti nella tabella `loan_lenders`, l'altro dividendo la quantità totale per il numero di lenders indicato dalla colonna `num_lenders_total`. Per lo svolgimento dei punti successivi, si prenderà in considerazione il primo, in quanto si riferisce direttamente ai lenders effettivamente presenti nel dataset.

In [42]:
lenders_amount=loanlen_amount[['lenders','amount']].groupby('lenders', as_index=False)['amount'].sum()
lenders_amount=pd.DataFrame(lenders_amount)
lenders_amount

,lenders,amount
0,000,1672.618411
1,00000,1380.693644
2,0002,2472.563566
3,00mike00,52.631579
4,0101craign0101,2623.565117
...,...,...
1639021,zzanita,87.500000
1639022,zzcyna7269,55.357143
1639023,zzinnia,38.000000
1639024,zzmcfate,2287.291955


In [43]:
lenders_amount_2=loanlen_amount[['lenders','amount_2']].groupby('lenders', as_index=False)['amount_2'].sum()
lenders_amount_2=pd.DataFrame(lenders_amount_2)
lenders_amount_2

,lenders,amount_2
0,000,1485.309656
1,00000,1249.947363
2,0002,2201.180463
3,00mike00,38.461538
4,0101craign0101,2424.088932
...,...,...
1639021,zzanita,62.500000
1639022,zzcyna7269,44.285714
1639023,zzinnia,32.758621
1639024,zzmcfate,2033.750197


### 9) For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

In [44]:
#punto 9
lenders

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
2349169,janet7309,Janet,NaN,NaN,NaN,1342097163,NaN,NaN,NaN,NaN,0
2349170,pj4198,NaN,NaN,NaN,NaN,1342097515,NaN,NaN,NaN,NaN,0
2349171,maria2141,Maria,NaN,NaN,US,1342099723,NaN,NaN,2.0,NaN,0
2349172,simone9846,Simone,NaN,NaN,NaN,1342100213,NaN,NaN,NaN,NaN,0


In [45]:
lenders.isnull().sum()

permanent_name             0
display_name            2768
city                 1619306
state                1713481
country_code         1458635
member_since               0
occupation           1844514
loan_because         2174852
loan_purchase_num     894281
invited_by           1852349
num_invited                0
dtype: int64

Dato che molti valori di ``country_code`` sono mancanti, assumo che la distribuzione della nazione di provenienza dei lenders sia la medesima di quelli per cui è nota. 

In [46]:
lenders['country_code'].value_counts()

US    591612
CA     67970
GB     38380
AU     37103
DE     16007
       ...  
IO         1
NU         1
GW         1
TF         1
KM         1
Name: country_code, Length: 234, dtype: int64

In [47]:
random.seed(123)
x = lenders['country_code'].value_counts(normalize=True)
missing = lenders['country_code'].isnull()
lenders.loc[missing,'country_code'] = np.random.choice(x.index, size=len(lenders[missing]),p=x.values)
lenders

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,CA,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,US,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,US,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,US,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,ES,1461301154,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
2349169,janet7309,Janet,NaN,NaN,US,1342097163,NaN,NaN,NaN,NaN,0
2349170,pj4198,NaN,NaN,NaN,US,1342097515,NaN,NaN,NaN,NaN,0
2349171,maria2141,Maria,NaN,NaN,US,1342099723,NaN,NaN,2.0,NaN,0
2349172,simone9846,Simone,NaN,NaN,GB,1342100213,NaN,NaN,NaN,NaN,0


Isolo il nome e la relativa nazione di appartenenza (espressa con il codice).

In [48]:
lenders_country = lenders[['permanent_name','country_code']]

In [49]:
lenders_country

,permanent_name,country_code
0,qian3013,CA
1,reena6733,US
2,mai5982,US
3,andrew86079135,US
4,nguyen6962,ES
...,...,...
2349169,janet7309,US
2349170,pj4198,US
2349171,maria2141,US
2349172,simone9846,GB


In [50]:
utf = lenders_amount.lenders.str.encode('utf-8')
utf

0                     b' 000'
1                   b' 00000'
2                    b' 0002'
3                b' 00mike00'
4          b' 0101craign0101'
                  ...        
1639021            b'zzanita'
1639022         b'zzcyna7269'
1639023            b'zzinnia'
1639024           b'zzmcfate'
1639025         b'zzrvmf8538'
Name: lenders, Length: 1639026, dtype: object

Prima di poter unire le tabelle `lenders_country` e `lenders_amount` è necessario applicare la funzione `str.strip`: aluni nomi dei lenders del dataset sono rappresentati con spazi bianchi all'inizio della stringa. Ciò altererebbe il risultato del join, che andrà fatto proprio su quella colonna.

In [51]:
lenders_amount.lenders = lenders_amount.lenders.str.strip()

In [52]:
len_country_amount = pd.merge(lenders_country, lenders_amount, left_on='permanent_name', right_on='lenders')
len_country_amount.drop('lenders', axis=1, inplace=True)

In [53]:
len_country_amount

,permanent_name,country_code,amount
0,qian3013,CA,46.839080
1,reena6733,US,417.948366
2,andrew86079135,US,349.601088
3,sirinapa6764,US,50.000000
4,rene7585,US,59.126984
...,...,...,...
1639021,carol8279,US,169.440389
1639022,eric91401545,US,63.780488
1639023,kate40761039,SE,36.280488
1639024,maria2141,US,110.507246


In [54]:
len_country_amount.isnull().sum()

permanent_name    0
country_code      0
amount            0
dtype: int64

Raggruppando per codice nazione, è possibile calcolare l'ammontare prestato da ogni stato.

In [55]:
country_lent = len_country_amount.groupby('country_code', as_index=False)['amount'].sum()

In [56]:
country_lent

,country_code,amount
0,AD,7.298314e+03
1,AE,1.980805e+06
2,AF,1.625091e+05
3,AG,1.017774e+03
4,AI,6.947933e+02
...,...,...
225,YE,7.108932e+04
226,YT,4.603746e+02
227,ZA,7.486100e+05
228,ZM,4.657092e+04


Come nel punto 5, ma utilizzando il codice nazione. Vi è una riga in meno rispetto all'esercizio citato precedentemente, dato che il codice dello stato Namibia è mancante (come si vedrà successivamente osservando il file `country_stats`).

In [57]:
country_borrowed = loans.groupby(['country_code'], as_index=False)['loan_amount'].sum()
country_borrowed

,country_code,loan_amount
0,AF,1967950.0
1,AL,4307350.0
2,AM,22950475.0
3,AZ,14784625.0
4,BA,477250.0
...,...,...
90,XK,3083025.0
91,YE,3444000.0
92,ZA,1006525.0
93,ZM,1978975.0


Si uniscono i due dataset. I valori mancanti sono imputati come degli 0, considerando che in mancanza di informazione uno stato non abbia ricevuto/effettuato prestiti.

In [58]:
quantità = pd.merge(country_lent, country_borrowed, how='outer', on = 'country_code')
quantità.columns = ['country_code', 'money_lent','money_borrowed']
quantità.fillna(0, inplace=True)

In [59]:
quantità

,country_code,money_lent,money_borrowed
0,AD,7.298314e+03,0.0
1,AE,1.980805e+06,0.0
2,AF,1.625091e+05,1967950.0
3,AG,1.017774e+03,0.0
4,AI,6.947933e+02,0.0
...,...,...,...
226,YT,4.603746e+02,0.0
227,ZA,7.486100e+05,1006525.0
228,ZM,4.657092e+04,1978975.0
229,ZW,1.154654e+04,5851875.0


In [60]:
quantità['difference'] = quantità['money_lent'] - quantità['money_borrowed']

In [61]:
quantità.sort_values('difference', ascending=False)

,country_code,money_lent,money_borrowed,difference
212,US,7.171820e+08,46352000.0,6.708300e+08
35,CA,9.660530e+07,50000.0,9.655530e+07
13,AU,6.130693e+07,0.0,6.130693e+07
72,GB,5.695232e+07,0.0,5.695232e+07
52,DE,3.600546e+07,0.0,3.600546e+07
...,...,...,...,...
107,KH,1.293341e+05,51613525.0,-5.148419e+07
169,PY,5.880376e+04,53964700.0,-5.390590e+07
105,KE,1.007956e+06,66735975.0,-6.572802e+07
158,PE,1.702506e+05,79437775.0,-7.926752e+07


Lo stato con la differenza più elevata tra denaro prestato e ricevuto sono gli USA, mentre quello con la più bassa le Filippine.

### 10) Which country has the highest ratio between the difference computed at the previous point and the population?

### 11) Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?


In [62]:
#punto 10 e 11 
country_stats = pd.read_csv("country_stats.csv")

In [63]:
country_stats

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh
...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Somalia,SO,SOM,Africa,Eastern Africa,14742523,NaN,NaN,NaN,NaN,NaN,NaN,Somalia
170,Central African Republic,CF,CAF,Africa,Middle Africa,4659080,NaN,0.352440,51.458,7.098980,4.230000,587.473961,Central African Republic
171,Samoa,WS,WSM,Oceania,Polynesia,196440,NaN,0.702000,NaN,NaN,NaN,NaN,Samoa
172,Palestine,PS,PS,Asia,Western Asia,4920724,NaN,0.677000,NaN,NaN,NaN,NaN,Palestine


In [64]:
country_stats.isnull().sum()

country_name                      0
country_code                      1
country_code3                     1
continent                         0
region                            0
population                        0
population_below_poverty_line    22
hdi                               3
life_expectancy                   6
expected_years_of_schooling       6
mean_years_of_schooling           6
gni                               6
kiva_country_name                 0
dtype: int64

In [65]:
country_stats[country_stats.isna().any(axis=1)] # la Namibia è l'unico stato con il codice mancante

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
115,Namibia,NaN,NAM,Africa,Southern Africa,2533794,28.7,0.640007,65.062,11.657589,6.676000,9769.848507,Namibia
151,Australia,AU,AUS,Oceania,Australia and New Zealand,24450561,NaN,0.938680,82.537,20.432720,13.175100,42822.196270,Australia
152,Bahrain,BH,BHR,Asia,Western Asia,1492584,NaN,0.823884,76.715,14.500000,9.419000,37236.363040,Bahrain
153,Brunei,BN,BRN,Asia,South-eastern Asia,428697,NaN,0.864937,79.019,14.944860,9.020000,72843.466090,Brunei
154,Cuba,CU,CUB,Americas,Caribbean,11484636,NaN,0.774669,79.573,13.937390,11.750781,7455.062477,Cuba
155,Cyprus,CY,CYP,Asia,Western Asia,1179551,NaN,0.855646,80.332,14.256130,11.695230,29458.522690,Cyprus
156,Finland,FI,FIN,Europe,Northern Europe,5523231,NaN,0.894517,81.006,17.047920,11.194000,38868.135520,Finland
157,Iceland,IS,ISL,Europe,Northern Europe,335025,NaN,0.921110,82.724,18.994220,12.190892,37065.214510,Iceland
158,Kuwait,KW,KWT,Asia,Western Asia,4136528,NaN,0.800183,74.549,13.260000,7.265369,76075.207350,Kuwait
159,Luxembourg,LU,LUX,Europe,Western Europe,583455,NaN,0.898465,81.881,13.863730,11.950833,62470.591290,Luxembourg


Calcolo la popolazione sopra la soglia di povertà.

In [66]:
country_stats['not_poors'] = (country_stats['population']/100) * (100-country_stats['population_below_poverty_line'])

In [67]:
country_stats

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,not_poors
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India,1.045900e+09
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria,5.726589e+07
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico,6.948984e+07
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan,1.388962e+08
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh,1.127988e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Somalia,SO,SOM,Africa,Eastern Africa,14742523,NaN,NaN,NaN,NaN,NaN,NaN,Somalia,NaN
170,Central African Republic,CF,CAF,Africa,Middle Africa,4659080,NaN,0.352440,51.458,7.098980,4.230000,587.473961,Central African Republic,NaN
171,Samoa,WS,WSM,Oceania,Polynesia,196440,NaN,0.702000,NaN,NaN,NaN,NaN,Samoa,NaN
172,Palestine,PS,PS,Asia,Western Asia,4920724,NaN,0.677000,NaN,NaN,NaN,NaN,Palestine,NaN


Si uniscono la tabella risultata dal punto precedente con quella contenente le informazioni sugli stati e la relativa popolazione.

In [68]:
country = pd.merge(quantità, country_stats, on='country_code')

Vengono quindi calcolati i due rapporti richiesti.

In [69]:
country['ratio_population'] = country['difference']/country['population']
country['ratio_not_poors'] = country['difference']/country['not_poors']

In [70]:
country.loc[country['ratio_population'].idxmax()]

country_code                                  NO
money_lent                             2.066e+07
money_borrowed                                 0
difference                             2.066e+07
country_name                              Norway
country_code3                                NOR
continent                                 Europe
region                           Northern Europe
population                               5305383
population_below_poverty_line                NaN
hdi                                     0.949423
life_expectancy                           81.711
expected_years_of_schooling              17.6719
mean_years_of_schooling                  12.7464
gni                                      67614.4
kiva_country_name                         Norway
not_poors                                    NaN
ratio_population                         3.89416
ratio_not_poors                              NaN
Name: 113, dtype: object

In [71]:
country.loc[country['ratio_not_poors'].idxmax()]

country_code                                   CA
money_lent                            9.66053e+07
money_borrowed                              50000
difference                            9.65553e+07
country_name                               Canada
country_code3                                 CAN
continent                                Americas
region                           Northern America
population                               36624199
population_below_poverty_line                 9.4
hdi                                      0.920284
life_expectancy                            82.224
expected_years_of_schooling                16.325
mean_years_of_schooling                   13.1051
gni                                       42581.9
kiva_country_name                          Canada
not_poors                             3.31815e+07
ratio_population                          2.63638
ratio_not_poors                           2.90991
Name: 24, dtype: object

Attraverso la funzione `idxmax` trovo gli stati con il rapporto maggiore. Al primo posto la Norvegia per la popolazione totale (risultato influenzato dal fatto che non sono registrati prestiti in entrata), mentre riguardo la popolazione sopra la soglia di povertà il Canada (per Norvegia il dato era mancante). 

In [72]:
country.sort_values('ratio_population', ascending=False)

,country_code,money_lent,money_borrowed,difference,country_name,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,not_poors,ratio_population,ratio_not_poors
113,NO,2.065999e+07,0.0,2.065999e+07,Norway,NOR,Europe,Northern Europe,5305383,NaN,0.949423,81.711,17.671870,12.746420,67614.353480,Norway,NaN,3.894156,NaN
24,CA,9.660530e+07,50000.0,9.655530e+07,Canada,CAN,Americas,Northern America,36624199,9.4,0.920284,82.224,16.325000,13.105115,42581.914330,Canada,3.318152e+07,2.636380,2.909912
7,AU,6.130693e+07,0.0,6.130693e+07,Australia,AUS,Oceania,Australia and New Zealand,24450561,NaN,0.938680,82.537,20.432720,13.175100,42822.196270,Australia,NaN,2.507383,NaN
71,IS,7.265353e+05,0.0,7.265353e+05,Iceland,ISL,Europe,Northern Europe,335025,NaN,0.921110,82.724,18.994220,12.190892,37065.214510,Iceland,NaN,2.168600,NaN
158,US,7.171820e+08,46352000.0,6.708300e+08,United States,USA,Americas,Northern America,324459463,15.1,0.919553,79.222,16.538150,13.217890,53245.076960,United States,2.754661e+08,2.067531,2.435254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,MN,7.666151e+04,15348375.0,-1.527171e+07,Mongolia,MNG,Asia,Eastern Asia,3075647,21.6,0.734832,69.806,14.845520,9.750000,10449.207270,Mongolia,2.411307e+06,-4.965366,-6.333375
144,SV,6.269675e+04,41691550.0,-4.162885e+07,El Salvador,SLV,Americas,Central America,6377853,34.9,0.679777,73.271,13.169930,6.532388,7732.025398,El Salvador,4.151982e+06,-6.527095,-10.026260
3,AM,6.809825e+04,22950475.0,-2.288238e+07,Armenia,ARM,Asia,Western Asia,2930450,32.0,0.742850,74.886,12.713218,11.291050,8189.115193,Armenia,1.992706e+06,-7.808486,-11.483067
126,PY,5.880376e+04,53964700.0,-5.390590e+07,Paraguay,PRY,Americas,South America,6811297,22.2,0.692966,73.004,12.318430,8.135100,8181.693782,Paraguay,5.299189e+06,-7.914190,-10.172480


In [73]:
country.sort_values('ratio_not_poors', ascending=False)

,country_code,money_lent,money_borrowed,difference,country_name,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,not_poors,ratio_population,ratio_not_poors
24,CA,9.660530e+07,50000.0,9.655530e+07,Canada,CAN,Americas,Northern America,36624199,9.4,0.920284,82.224,16.325000,13.105115,42581.914330,Canada,3.318152e+07,2.636380,2.909912
158,US,7.171820e+08,46352000.0,6.708300e+08,United States,USA,Americas,Northern America,324459463,15.1,0.919553,79.222,16.538150,13.217890,53245.076960,United States,2.754661e+08,2.067531,2.435254
135,SE,1.626307e+07,0.0,1.626307e+07,Sweden,SWE,Europe,Northern Europe,9910701,15.0,0.912706,82.347,16.055459,12.273700,46250.793820,Sweden,8.424096e+06,1.640961,1.930542
112,NL,2.711827e+07,0.0,2.711827e+07,Netherlands,NLD,Europe,Western Europe,17035938,8.8,0.924312,81.706,18.116540,11.914070,46325.576580,Netherlands,1.553678e+07,1.591827,1.745425
27,CH,1.120694e+07,0.0,1.120694e+07,Switzerland,CHE,Europe,Western Europe,8476005,6.6,0.939131,83.133,16.040410,13.370000,56363.957800,Switzerland,7.916589e+06,1.322196,1.415627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,SA,1.045613e+06,0.0,1.045613e+06,Saudi Arabia,SAU,Asia,Western Asia,32938213,NaN,0.846602,74.444,16.112270,9.627520,51320.113230,Saudi Arabia,NaN,0.031745,NaN
133,SB,3.324230e+03,815575.0,-8.122508e+05,Solomon Islands,SLB,Oceania,Melanesia,611343,NaN,0.514999,68.107,9.610000,5.330584,1560.858813,Solomon Islands,NaN,-1.328633,NaN
136,SG,3.502015e+06,0.0,3.502015e+06,Singapore,SGP,Asia,South-eastern Asia,5708844,NaN,0.924866,83.209,15.400000,11.572410,78162.323870,Singapore,NaN,0.613437,NaN
141,SO,5.455206e+03,308725.0,-3.032698e+05,Somalia,SOM,Africa,Eastern Africa,14742523,NaN,NaN,NaN,NaN,NaN,NaN,Somalia,NaN,-0.020571,NaN


### 12) For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

In [74]:
#punto 12
loans

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,disburse_date,planned_expiration_date,duration,disburse_year
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,to buy additional inventories for her sari-sa...,PH,Philippines,Ozamiz - Ozamiz City,shared,0.1,PHP,126.0,2014-01-15 02:23:45.000 +0000,2014-02-14 03:30:06+00:00,2013-12-22 08:00:00+00:00,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner,2013-12-22,2014-02-14,54 days,2013.0
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,"To buy American clothing such as blouses, shir...",HN,Honduras,"La Lopez, Choloma",shared,0.1,HNL,201.0,2014-01-14 20:23:20.000 +0000,2014-03-26 22:25:07+00:00,2013-12-20 08:00:00+00:00,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner,2013-12-20,2014-03-26,96 days,2013.0
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,"to buy stock of rice, sugar and flour",PK,Pakistan,Lala Musa,shared,0.1,PKR,245.0,2014-01-16 11:32:58.000 +0000,2014-02-15 21:10:05+00:00,2014-01-09 08:00:00+00:00,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,2014-01-09,2014-02-15,37 days,2014.0
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,"to buy cucumber and tomato seeds, as well as f...",KG,Kyrgyzstan,"Aravan village, Osh region",shared,0.1,KGS,171.0,2014-01-20 09:59:48.000 +0000,2014-02-21 03:10:02+00:00,2014-01-17 08:00:00+00:00,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner,2014-01-17,2014-02-21,35 days,2014.0
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,to buy organic fertilizer and agrochemical pr...,PH,Philippines,"Baleleng, Sto. Thomas, Isabela",shared,0.1,PHP,123.0,2014-01-14 05:46:21.000 +0000,2014-02-13 06:10:02+00:00,2013-12-17 08:00:00+00:00,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,2013-12-17,2014-02-13,58 days,2013.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419602,988180,NaN,NaN,NaN,NaN,400.0,400.0,funded,Tailoring,Services,NaN,KE,Kenya,NaN,shared,0.1,KES,138.0,2015-12-01 04:49:50.000 +0000,2016-01-02 01:00:03+00:00,2015-11-23 08:00:00+00:00,2015-12-28 15:44:18.000 +0000,14.0,16,4,2,"#Parent, #Repeat Borrower, #Woman Owned Biz",NaN,NaN,monthly,field_partner,2015-11-23,2016-01-02,40 days,2015.0
1419603,988213,Perlita,English,"Perlita is 52 years old, married and has three...","Perlita is 52 years old, married and has three...",300.0,300.0,funded,Pigs,Agriculture,to buy feeds and other supplies to raise her pigs,PH,Philippines,"Numancia, Aklan",shared,0.1,PHP,145.0,2015-12-01 05:13:30.000 +0000,2016-01-02 16:40:07+00:00,2015-11-24 08:00:00+00:00,2015-12-22 10:37:06.000 +0000,14.0,12,1,1,"#Animals, #Elderly, #Repeat Borrower, #Woman O...",female,true,irregular,field_partner,2015-11-24,2016-01-02,39 days,2015.0
1419604,989109,Okyeso Nyame Group,English,Okyeso Nyame group will begin its third cycle ...,Okyeso Nyame group will begin its third cycle ...,2425.0,2425.0,funded,Ba

Estraggo gli anni in cui sono stati effettuati i prestiti, dopodichè identifico quelli bisestili (nel range presente nel database sono 2008, 2012 e 2016).

In [75]:
loans['disburse_time'].dt.year.unique()

array([2013., 2014., 2015., 2012., 2016., 2010., 2017., 2018., 2009.,
       2011., 2007.,   nan, 2006., 2008., 2005.])

In [76]:
loans['planned_expiration_time'].dt.year.unique()

array([2014., 2015., 2012., 2016.,   nan, 2017., 2013., 2018.])

In [77]:
bisestili = {2008, 2012, 2016}

Per trovare la quantità di denaro prestata in riferimento ad ogni anno serviranno: inzio, fine, l'ammontare e la durata dei prestiti. Si è scelto di non prendere in considerazione i casi in cui `disburse_time` sia superiore a  `planned_expiration_time`. 

In [78]:
anno1 = [anno.year for anno in loans['disburse_time']]

In [79]:
anno2 = [anno.year for anno in loans['planned_expiration_time']]

In [80]:
amounts = list(loans['loan_amount'])

In [81]:
durata = loans['duration']

Scorro tutte le righe del dataset, se la durata del prestito non è negativa si identificano due possibilità: l'anno dell'esborso è lo stesso della data di scadenza, `disburse_time` e `planned_expiration_time` cadono in anni differenti. Nel primo caso l'importo viene ripartito totalmente all'anno in corso, nel secondo viene creata una lista contenente tutto il range di anni del prestito e l'ammontare viene distribuito proporzionalmente al numero di giorni.

In [82]:
lanno=[]
limporto=[]

In [83]:
for i in range(len(loans)):
    importo=amounts[i]
    anno_in=anno1[i]
    anno_fin=anno2[i]
    giorni_prestito=durata[i].days
    if giorni_prestito >= 0:
        if anno_in==anno_fin:
            lanno.append(anno_in)
            limporto.append(importo)
        else:
            anni=list(range(anno_in,anno_fin+1))  
    
            anno=anni[0]
            if anno in bisestili:
                ga=366-loans['disburse_time'][i].dayofyear
            else:    
                ga=365-loans['disburse_time'][i].dayofyear
            ia=(ga*importo)/giorni_prestito
            lanno.append(anno)
            limporto.append(ia)

            for j in range(1,len(anni)-1):
                anno=anni[j]
                if anno in bisestili:
                    ia=(366*importo)/giorni_prestito
                else:
                    ia=(365*importo)/giorni_prestito
                lanno.append(anno)
                limporto.append(ia)

            anno=anni[-1]
            ga=loans['planned_expiration_time'][i].dayofyear
            ia=(ga*importo)/giorni_prestito
            lanno.append(anno)
            limporto.append(ia)
    

In [84]:
anno_importo = pd.DataFrame()
anno_importo['Anno'] = lanno
anno_importo['Importo'] = limporto

In [85]:
pd.DataFrame(anno_importo.groupby('Anno')['Importo'].sum())

,Importo
Anno,
2011,6.299118e+05
2012,1.130523e+08
2013,1.235038e+08
2014,1.506786e+08
2015,1.552877e+08
2016,1.542358e+08
2017,1.657267e+08
2018,5.079946e+06


In [86]:
end = time.time()

print("Durata: {} minuti".format(int((end-start)/60)))

Durata: 4 minuti
